In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [2]:
datasets = [
    ["abalone_input.pd","abalone_target.pd"], 
    ["adult_input.pd","adult_target.pd"], # Large
    ["banknote_input.pd","banknote_target.pd"], # Easy
    ["bank_input.pd","bank_target.pd"], # Large
    ["breast_cancer_input.np","breast_cancer_target.np"],
    ["cars_input.pd","cars_target.pd"], 
    ["contraceptive_input.pd","contraceptive_target.pd"], 
    ["generated6_input.np","generated6_target.np"],
    ["hrss_input.pd","hrss_target.pd"], # Large
    ["iris_input.pd","iris_target.pd"],
    ["steel_input.pd","steel_target.pd"],
    ["students_input.pd","students_target.pd"],
    #["sensorless_input.pd","sensorless_target.pd"], # Very Large dataset
]

In [3]:
gate_reduction_methods = [
    #"feature_importance",
    #"feature_importance_lda",
    #"feature_importance_lda_max",
    #"feature_importance_lr",
    #"feature_importance_lr_max",
    #"feature_importance_xgb",
    "feature_importance_pca_loadings",
    #"PCA",
    #None
]

In [4]:
parameters = {
    "X": None,
    "y": None,
    "n_experts": 3,
    "iterations": 100,
    "max_depth": 1,
    "init_learning_rate": 110,
    "learning_rate_decay": 0.985,
    "initialization_method": "random",
    "feature_names": None,
    "class_names": None,
    "use_2_dim_gate_based_on": "overwritten",
    "use_2_dim_clustering": False,
    "black_box_algorithm": None,
    "verbose": False
    }
parameters_fit = {
    "optimization_method": "least_squares_linear_regression",
    "early_stopping": "likelihood",
    }

In [5]:
start = timer()
runs = 50
rows = []
for dataset in datasets:
    print("Starting",dataset[0],"...")
    data_input = pickle.load(open("../datasets/" + dataset[0], "rb"))
    data_target = pickle.load(open("../datasets/" + dataset[1], "rb"))
    
    use_dataframe = False
    if isinstance(data_input, pd.core.frame.DataFrame):
        use_dataframe = True
        
      
    dimensionality_reduction = gate_reduction_methods
    dict_results = {
        "dataset" : dataset[0],
        "n_features" : data_input.shape[1]
    } 
    
    for method in dimensionality_reduction:
        print("Starting",method,"...")
        parameters["use_2_dim_gate_based_on"] = method

        train_accuracies = []
        val_accuracies = []
        rkf = RepeatedKFold(n_splits=4, n_repeats=runs)
        for train_idx, val_idx in rkf.split(data_input):
            if use_dataframe:
                X_temp = data_input.iloc[train_idx]
                y_temp = data_target.iloc[train_idx]
                X_temp.reset_index(inplace=True, drop=True)
                y_temp.reset_index(inplace=True, drop=True)
            else:
                X_temp = data_input[train_idx]
                y_temp = data_target[train_idx]

            parameters["X"] = X_temp
            parameters["y"] = y_temp
            parameters["initialization_method"] = Random_init(seed=None)
            modt = MoDT(**parameters)
            modt.fit(**parameters_fit)
            train_accuracies.append(modt.score_internal_disjoint())

            if use_dataframe:
                X_temp = data_input.iloc[val_idx]
                y_temp = data_target.iloc[val_idx]
                X_temp.reset_index(inplace=True, drop=True)
                y_temp.reset_index(inplace=True, drop=True)
            else:
                X_temp = data_input[val_idx]
                y_temp = data_target[val_idx]
            val_accuracies.append(modt.score(X_temp, y_temp))

        train_accuracy = np.mean(train_accuracies)
        val_accuracy = np.mean(val_accuracies)
        train_std = np.std(train_accuracies)
        val_std = np.std(val_accuracies)
        dict_results[str(method) + "_train"] = train_accuracy
        dict_results[str(method) + "_test"] = val_accuracy
        dict_results[str(method) + "_train_std"] = train_std
        dict_results[str(method) + "_test_std"] = val_std
        
    rows.append(dict_results)
    
print("Duration", timer() - start)
df_performance = pd.DataFrame(rows)

Starting abalone_input.pd ...
Starting feature_importance_pca_loadings ...
Starting adult_input.pd ...
Starting feature_importance_pca_loadings ...
Starting banknote_input.pd ...
Starting feature_importance_pca_loadings ...
Starting bank_input.pd ...
Starting feature_importance_pca_loadings ...
Starting breast_cancer_input.np ...
Starting feature_importance_pca_loadings ...
Starting cars_input.pd ...
Starting feature_importance_pca_loadings ...
Starting contraceptive_input.pd ...
Starting feature_importance_pca_loadings ...
Starting generated6_input.np ...
Starting feature_importance_pca_loadings ...
Starting hrss_input.pd ...
Starting feature_importance_pca_loadings ...
Starting iris_input.pd ...
Starting feature_importance_pca_loadings ...
Starting steel_input.pd ...
Starting feature_importance_pca_loadings ...
Starting students_input.pd ...
Starting feature_importance_pca_loadings ...
Duration 4370.7589062


In [6]:
df_performance

,dataset,n_features,feature_importance_pca_loadings_train,feature_importance_pca_loadings_test,feature_importance_pca_loadings_train_std,feature_importance_pca_loadings_test_std
0,abalone_input.pd,8,0.688417,0.678798,0.007741,0.014625
1,adult_input.pd,14,0.765467,0.762486,0.022887,0.023957
2,banknote_input.pd,4,0.954840,0.941370,0.008545,0.013280
3,bank_input.pd,20,0.897610,0.897634,0.001426,0.002810
4,breast_cancer_input.np,10,0.943585,0.906927,0.012001,0.024078
5,cars_input.pd,6,0.762743,0.759167,0.027107,0.030483
6,contraceptive_input.pd,9,0.453958,0.441100,0.008763,0.022696
7,generated6_input.np,2,0.856180,0.852696,0.056522,0.058880
8,hrss_input.pd,18,0.763603,0.763327,0.002539,0.005183
9,iris_input.pd,4,0.975468,0.937667,0.011455,0.040062


In [8]:
pickle.dump(df_performance, open("dataframes/ex4b_df_selection_performances_loadings.pd", "wb"))

In [9]:
df = pd.read_pickle("dataframes/ex4b_df_selection_performances.pd")

In [11]:
df["feature_importance_pca_loadings_train"] = df_performance["feature_importance_pca_loadings_train"]
df["feature_importance_pca_loadings_test"] = df_performance["feature_importance_pca_loadings_test"]
df["feature_importance_pca_loadings_train_std"] = df_performance["feature_importance_pca_loadings_train_std"]
df["feature_importance_pca_loadings_test_std"] = df_performance["feature_importance_pca_loadings_test_std"]



In [19]:
df["feature_importance_pca_loadings_test"]

0     0.678798
1     0.762486
2     0.941370
3     0.897634
4     0.906927
5     0.759167
6     0.441100
7     0.852696
8     0.763327
9     0.937667
10    0.531590
11    0.443963
Name: feature_importance_pca_loadings_test, dtype: float64

In [21]:
pickle.dump(df, open("dataframes/ex4b_df_selection_performances.pd", "wb"))